# Load Data từ Bronze Layer sang Silver Layer

Notebook này sẽ đọc dữ liệu từ Bronze layer (MinIO) và xử lý để load vào các bảng Iceberg trong Silver layer với Nessie catalog.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import csv, io, os, re
from datetime import datetime
from typing import Dict

# Cấu hình AWS/MinIO credentials
os.environ.update({
    'AWS_REGION': 'us-east-1',
    'AWS_ACCESS_KEY_ID': 'admin',
    'AWS_SECRET_ACCESS_KEY': 'admin123'
})

# Khởi tạo Spark Session với Nessie Catalog
spark = (
    SparkSession.builder
    .appName("Load_Bronze_To_Silver")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    # Nessie Catalog
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # S3/MinIO Config
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # Hadoop S3A Config
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Executor Environment
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # Local JAR files
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.silver_tables")
spark.sql("USE nessie.silver_tables")
print(f"✅ Spark Session initialized | Master: {spark.sparkContext.master} | App ID: {spark.sparkContext.applicationId}")


25/12/02 14:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/02 14:08:53 WARN DependencyUtils: Local jar /opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar does not exist, skipping.
25/12/02 14:08:54 INFO SparkContext: Running Spark version 3.5.5
25/12/02 14:08:54 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/12/02 14:08:54 INFO SparkContext: Java version 17.0.14
25/12/02 14:08:54 INFO ResourceUtils: ==============================================================
25/12/02 14:08:54 INFO ResourceUtils: No custom resources configured for spark.driver.
25/12/02 14:08:54 INFO ResourceUtils: ==============================================================
25/12/02 14:08:54 INFO SparkContext: Submitted application: Load_Bronze_To_Silver
25/12/02 14:08:54 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 2, script: , vend

✅ Spark Session initialized | Master: spark://spark-master:7077 | App ID: app-20251202140855-0000


In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from pyspark.sql.window import Window
# from datetime import datetime
# import os
# import hashlib  # Để dùng trong batch processing nếu cần
# import csv
# import io


# # Khởi tạo Spark Session với Iceberg và Nessie catalog
# spark = (
#     SparkSession.builder
#     # .master("spark://spark-master:7077") # để chạy DAG bên Spark Cluster
#     .appName("Load_Bronze_To_Silver")
#     .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
#     .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
#     .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
#     .config("spark.sql.catalog.nessie.ref", "main")
#     .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
#     .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
#     .config("spark.sql.catalog.nessie.s3.access-key", "admin")
#     .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
#     .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
#     .config("spark.hadoop.fs.s3a.access.key", "admin")
#     .config("spark.hadoop.fs.s3a.secret.key", "admin123")
#     .config("spark.hadoop.fs.s3a.path.style.access", "true")
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .getOrCreate()
# )
# spark.sparkContext.setLogLevel("ERROR")
# print("Spark Session đã được khởi tạo với Nessie catalog!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 05:20:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session đã được khởi tạo với Nessie catalog!


## 2. Load Bảng SCHOOL

In [2]:
print("=" * 80)
print("LOAD BẢNG SCHOOL")
print("=" * 80)

# Đọc và merge tất cả các năm
years = [2021, 2022, 2023, 2024, 2025]
base_path = "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_"
df_school = spark.read.option("header", "true").option("inferSchema", "true").csv([f"{base_path}{year}.csv" for year in years]).select("TenTruong", "MaTruong", "TinhThanh").dropDuplicates()

# Transform
df_school_silver = df_school.select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("TenTruong").cast("string").alias("schoolName"),
    col("TinhThanh").cast("string").alias("province"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("schoolId").isNotNull() & col("schoolName").isNotNull())

# Ghi vào Silver
df_school_silver.writeTo("nessie.silver_tables.school").using("iceberg").createOrReplace()
print(f"Đã ghi {df_school_silver.count()} dòng vào school")

# Verify
spark.table("nessie.silver_tables.school").show(5, truncate=False)

LOAD BẢNG SCHOOL


Đã ghi 265 dòng vào school
+--------+--------------------------------------+-----------+--------------------------+--------------------------+
|schoolId|schoolName                            |province   |created_at                |updated_at                |
+--------+--------------------------------------+-----------+--------------------------+--------------------------+
|DHF     |Đại học Ngoại Ngữ - Đại học Huế       |Huế        |2025-12-01 15:39:04.602019|2025-12-01 15:39:04.602019|
|DVB     |Đại học Việt Bắc                      |Thái Nguyên|2025-12-01 15:39:04.602019|2025-12-01 15:39:04.602019|
|DCQ     |Đại học Công Nghệ và Quản Lý Hữu Nghị |Hà Nội     |2025-12-01 15:39:04.602019|2025-12-01 15:39:04.602019|
|NTT     |Đại học Nguyễn Tất Thành              |TP HCM     |2025-12-01 15:39:04.602019|2025-12-01 15:39:04.602019|
|KGH     |Trường Sĩ Quan Không Quân - Hệ Đại học|Khánh Hòa  |2025-12-01 15:39:04.602019|2025-12-01 15:39:04.602019|
+--------+-----------------------------------

## 3. Load Bảng MAJOR

In [3]:
from pyspark.sql.functions import col, lower, trim, regexp_replace, current_timestamp

df_major = spark.read.option("header", "true") \
    .option("inferSchema", "false") \
    .option("encoding", "UTF-8") \
    .csv("s3a://bronze/structured_data/danh sách các ngành đại học/Danh_sách_các_ngành.csv")

df_major_clean = df_major.select(
    regexp_replace(trim(col(df_major.columns[0])).cast("string"), r"\.0$", "").alias("majorId"),
    trim(col(df_major.columns[1])).cast("string").alias("majorName")
).filter(
    (col("majorId").isNotNull()) &
    (col("majorName").isNotNull()) &
    (col("majorId") != "") &
    (col("majorName") != "") &
    (lower(col("majorId")) != "nan")
)

# Chuẩn hoá để dedupe theo lowercase
df_major_silver = df_major_clean \
    .withColumn("majorId_lower", lower(col("majorId"))) \
    .dropDuplicates(["majorId_lower"]) \
    .select(
        col("majorId"),
        col("majorName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )

df_major_silver.writeTo("nessie.silver_tables.major").using("iceberg").createOrReplace()

print(f"Đã ghi {df_major_silver.count()} dòng vào major")
spark.table("nessie.silver_tables.major").show(5, truncate=False)


Đã ghi 3085 dòng vào major
+-------+------------------------------------------------------------+-------------------------+-------------------------+
|majorId|majorName                                                   |created_at               |updated_at               |
+-------+------------------------------------------------------------+-------------------------+-------------------------+
|106    |Khoa học Máy tính                                           |2025-12-01 15:39:08.82324|2025-12-01 15:39:08.82324|
|107    |Kỹ thuật Máy tính                                           |2025-12-01 15:39:08.82324|2025-12-01 15:39:08.82324|
|108    |Điện - Điện tử - Viễn Thông - Tự động hoá - Thiết kế vi mạch|2025-12-01 15:39:08.82324|2025-12-01 15:39:08.82324|
|109    |Kỹ Thuật Cơ khí                                             |2025-12-01 15:39:08.82324|2025-12-01 15:39:08.82324|
|110    |Kỹ Thuật Cơ Điện tử                                         |2025-12-01 15:39:08.82324|2025-12-01 15:39

## 4. Load Bảng SUBJECT_GROUP và SUBJECT

In [4]:
print("=" * 80)
print("LOAD BẢNG SUBJECT_GROUP và SUBJECT")
print("=" * 80)

# Đọc file tohop_mon_fixed.csv
df_tohop = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/tohop_mon_fixed.csv")

# --- SUBJECT_GROUP ---
df_subject_group_silver = df_tohop.select(
    col(df_tohop.columns[0]).cast("int").alias("subjectGroupId"),
    col(df_tohop.columns[1]).cast("string").alias("subjectGroupName"),
    col(df_tohop.columns[2]).cast("string").alias("subjectCombination"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("subjectGroupId").isNotNull() & col("subjectGroupName").isNotNull() & col("subjectCombination").isNotNull()).dropDuplicates(["subjectGroupName", "subjectCombination"])
df_subject_group_silver.writeTo("nessie.silver_tables.subject_group").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_group_silver.count()} dòng vào subject_group")

# --- SUBJECT ---
df_subject = (
    df_tohop.select(explode(split(col(df_tohop.columns[2]), "-")).alias("subjectName"))
            .withColumn("subjectName", trim(col("subjectName")))
            .filter(col("subjectName").isNotNull() & (col("subjectName") != ""))
            .withColumn("subjectName_lower", lower(col("subjectName")))
            # loại bỏ trùng theo chữ thường
            .dropDuplicates(["subjectName_lower"])
)

window_spec = Window.orderBy("subjectName_lower")
df_subject_silver = df_subject.withColumn("subjectId", row_number().over(window_spec)).select(
    col("subjectId").cast("int"),
    col("subjectName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_subject_silver.writeTo("nessie.silver_tables.subject").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_silver.count()} dòng vào subject")

# Verify
spark.table("nessie.silver_tables.subject_group").orderBy("subjectGroupId").show(5, truncate=False)
spark.table("nessie.silver_tables.subject").show(5, truncate=False)

LOAD BẢNG SUBJECT_GROUP và SUBJECT
Đã ghi 232 dòng vào subject_group
Đã ghi 51 dòng vào subject
+--------------+----------------+------------------+--------------------------+--------------------------+
|subjectGroupId|subjectGroupName|subjectCombination|created_at                |updated_at                |
+--------------+----------------+------------------+--------------------------+--------------------------+
|1             |A00             |Toán-Lí-Hóa       |2025-12-01 15:39:10.880774|2025-12-01 15:39:10.880774|
|2             |A01             |Toán-Lí-Ngoại ngữ |2025-12-01 15:39:10.880774|2025-12-01 15:39:10.880774|
|3             |A02             |Toán-Lí-Sinh      |2025-12-01 15:39:10.880774|2025-12-01 15:39:10.880774|
|4             |A03             |Toán-Lí-Sử        |2025-12-01 15:39:10.880774|2025-12-01 15:39:10.880774|
|5             |A04             |Toán-Lí-Địa       |2025-12-01 15:39:10.880774|2025-12-01 15:39:10.880774|
+--------------+----------------+---------------

## 5. Load Bảng SELECTION_METHOD

In [5]:
print("=" * 80)
print("LOAD BẢNG SELECTION_METHOD")
print("=" * 80)

# Đọc từ file benchmark để lấy các phương thức xét tuyển
df_benchmark = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")

# Lấy PhuongThuc và loại bỏ "năm ..."
df_selection = df_benchmark.select(trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")).alias("selectionMethodName")).filter(col("selectionMethodName").isNotNull() & (col("selectionMethodName") != "")).distinct()

window_spec = Window.orderBy("selectionMethodName")
df_selection_method_silver = df_selection.withColumn("selectionMethodId", row_number().over(window_spec)).select(
    col("selectionMethodId").cast("int"),
    col("selectionMethodName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_selection_method_silver.writeTo("nessie.silver_tables.selection_method").using("iceberg").createOrReplace()
print(f"Đã ghi {df_selection_method_silver.count()} dòng vào selection_method")

# Verify
spark.table("nessie.silver_tables.selection_method").show(10, truncate=False)

LOAD BẢNG SELECTION_METHOD
Đã ghi 10 dòng vào selection_method
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|selectionMethodId|selectionMethodName                                   |created_at                |updated_at                |
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|1                |Điểm chuẩn theo phương thức Điểm học bạ               |2025-12-01 15:39:14.369252|2025-12-01 15:39:14.369252|
|2                |Điểm chuẩn theo phương thức Điểm thi THPT             |2025-12-01 15:39:14.369252|2025-12-01 15:39:14.369252|
|3                |Điểm chuẩn theo phương thức Điểm xét tuyển kết hợp    |2025-12-01 15:39:14.369252|2025-12-01 15:39:14.369252|
|4                |Điểm chuẩn theo phương thức Điểm xét tốt nghiệp THPT  |2025-12-01 15:39:14.369252|2025-12-01 15:39:14.369252|
|5                |Điểm chuẩn theo

## 6. Load Bảng GradingScale

In [6]:
print("=" * 80)
print("LOAD BẢNG GRADING_SCALE TỪ PHANLOAITHANGDIEM")
print("=" * 80)

# 1. Đọc dữ liệu gốc từ file CSV (giống benchmark)
df_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("encoding", "UTF-8")
        .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
)

# 2. Lấy unique PhanLoaiThangDiem
df_grading_raw = (
    df_raw
        .select(trim(col("PhanLoaiThangDiem")).alias("description"))
        .filter(col("description").isNotNull() & (col("description") != ""))
        .dropDuplicates(["description"])
)

# 3. Tách giá trị số trong description làm "value" (nếu có, vd: "thang 40" -> 40)
df_grading = (
    df_grading_raw
        .withColumn(
            "value",
            regexp_extract(col("description"), r"(\d+(?:\.\d+)?)", 1).cast("float")
        )
        .withColumn("gradingScaleId", monotonically_increasing_id().cast("int"))
        .withColumn("created_at", current_timestamp())
        .withColumn("updated_at", current_timestamp())
        .select(
            "gradingScaleId",
            "value",
            "description",
            "created_at",
            "updated_at"
        )
)

# 4. Ghi vào bảng Iceberg grading_scale đã tạo trước đó
df_grading.writeTo("nessie.silver_tables.grading_scale") \
          .using("iceberg") \
          .createOrReplace()

print(f"Đã ghi {df_grading.count()} dòng vào grading_scale")

# 5. Verify
spark.table("nessie.silver_tables.grading_scale").show(truncate=False)


LOAD BẢNG GRADING_SCALE TỪ PHANLOAITHANGDIEM
Đã ghi 10 dòng vào grading_scale
+--------------+------+---------------+--------------------------+--------------------------+
|gradingScaleId|value |description    |created_at                |updated_at                |
+--------------+------+---------------+--------------------------+--------------------------+
|0             |30.0  |Thang điểm 30  |2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|1             |1200.0|Thang điểm 1200|2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|2             |40.0  |Thang điểm 40  |2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|3             |50.0  |Thang điểm 50  |2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|4             |150.0 |Thang điểm 150 |2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|5             |10.0  |Thang điểm 10  |2025-12-01 15:39:16.761752|2025-12-01 15:39:16.761752|
|6             |100.0 |Thang điểm 100 |2025-12-01 15:39:16.761752|2025-12-01

## 6. Load Bảng BENCHMARK

In [11]:
from pyspark.sql.functions import (
    col, trim, regexp_replace, current_timestamp,
    avg, round, expr
)

print("=" * 80)
print("LOAD BẢNG BENCHMARK")
print("=" * 80)

# =========================
# 1. ĐỌC & CHUẨN HÓA DỮ LIỆU BRONZE
# =========================

df_benchmark = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("encoding", "UTF-8")
    .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
)

# Chuẩn hóa cột PhuongThuc: bỏ phần "năm XXXX ..."
df_benchmark = df_benchmark.withColumn(
    "PhuongThuc_cleaned",
    trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", ""))
)

# Lookup tables từ Silver
df_selection_lookup     = spark.table("nessie.silver_tables.selection_method")
df_subject_group_lookup = spark.table("nessie.silver_tables.subject_group")
df_grading_scale_lookup = spark.table("nessie.silver_tables.grading_scale")

# Join lookup + chuẩn hóa
df_benchmark_base = (
    df_benchmark
    .join(
        df_selection_lookup,
        df_benchmark["PhuongThuc_cleaned"] == df_selection_lookup["selectionMethodName"],
        "left"
    )
    .join(
        df_subject_group_lookup,
        df_benchmark["KhoiThi"] == df_subject_group_lookup["subjectGroupName"],
        "left"
    )
    .join(
        df_grading_scale_lookup,
        trim(df_benchmark["PhanLoaiThangDiem"]) == df_grading_scale_lookup["description"],
        "left"
    )
    .select(
        col("MaTruong").cast("string").alias("schoolId"),
        col("MaNganh").cast("string").alias("majorId"),
        col("subjectGroupId").cast("int"),
        col("selectionMethodId").cast("int"),
        col("gradingScaleId").cast("int"),
        col("Nam").cast("int").alias("year"),
        col("DiemChuan").cast("double").alias("score"),
    )
    .filter(
        col("schoolId").isNotNull() &
        col("majorId").isNotNull() &
        col("gradingScaleId").isNotNull() &
        col("year").isNotNull() &
        col("score").isNotNull() &
        col("selectionMethodId").isNotNull()
        # col("subjectGroupId").isNotNull()  # nếu muốn bắt buộc khối thi thì mở dòng này
    )
    .dropDuplicates([
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "year",
        "gradingScaleId",
        "score"
    ])
)

# =========================
# 2. GROUP BY & LẤY ĐIỂM TRUNG BÌNH
# =========================

df_benchmark_grouped = (
    df_benchmark_base
    .groupBy(
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "gradingScaleId",
        "year"
    )
    .agg(
        round(avg("score"), 2).alias("score")
    )
)

table_name = "nessie.silver_tables.benchmark"

# =========================
# 3. CHECK BẢNG SILVER ĐÃ TỒN TẠI CHƯA
# =========================

try:
    spark.table(table_name)
    table_exists = True
    print(f"Bảng {table_name} đã tồn tại → dùng MERGE (upsert).")
except Exception:
    table_exists = False
    print(f"Bảng {table_name} chưa tồn tại → tạo mới full-load.")

# =========================
# 4. LẦN ĐẦU: TẠO BẢNG FULL (DÙNG xxhash64 LÀM benchmarkId)
# =========================

if not table_exists:
    df_benchmark_silver = (
        df_benchmark_grouped
        .withColumn(
            "benchmarkId",
            expr(
                """
                CAST(
                    xxhash64(
                        schoolId,
                        majorId,
                        COALESCE(subjectGroupId, -1),
                        selectionMethodId,
                        gradingScaleId,
                        year
                    ) AS BIGINT
                )
                """
            )
        )
        .withColumn("created_at", current_timestamp())
        .withColumn("updated_at", current_timestamp())
        .select(
            "benchmarkId",
            "schoolId",
            "majorId",
            "subjectGroupId",
            "selectionMethodId",
            "gradingScaleId",
            "year",
            "score",
            "created_at",
            "updated_at"
        )
    )

    df_benchmark_silver.writeTo(table_name).using("iceberg").createOrReplace()
    print(f"Đã tạo mới benchmark với {df_benchmark_silver.count()} dòng")

# =========================
# 5. CÁC LẦN SAU: MERGE / UPSERT
# =========================

else:
    # Staging từ bronze sau khi chuẩn hóa + group
    df_staging = (
        df_benchmark_grouped
        .withColumn("created_at", current_timestamp())
        .withColumn("updated_at", current_timestamp())
    )

    df_staging.createOrReplaceTempView("benchmark_staging")

    # MERGE:
    # - MATCHED: update score + updated_at
    # - NOT MATCHED: insert bản ghi mới với benchmarkId = hash(business key)
    spark.sql(f"""
        MERGE INTO {table_name} AS t
        USING benchmark_staging AS s
        ON  t.schoolId          = s.schoolId
        AND t.majorId           = s.majorId
        AND COALESCE(t.subjectGroupId,  -1) = COALESCE(s.subjectGroupId,  -1)
        AND t.selectionMethodId = s.selectionMethodId
        AND t.gradingScaleId    = s.gradingScaleId
        AND t.year              = s.year

        WHEN MATCHED THEN UPDATE SET
            t.score      = s.score,
            t.updated_at = current_timestamp()

        WHEN NOT MATCHED THEN INSERT (
            benchmarkId,
            schoolId,
            majorId,
            subjectGroupId,
            selectionMethodId,
            gradingScaleId,
            year,
            score,
            created_at,
            updated_at
        ) VALUES (
            CAST(
                xxhash64(
                    s.schoolId,
                    s.majorId,
                    COALESCE(s.subjectGroupId, -1),
                    s.selectionMethodId,
                    s.gradingScaleId,
                    s.year
                ) AS BIGINT
            ),
            s.schoolId,
            s.majorId,
            s.subjectGroupId,
            s.selectionMethodId,
            s.gradingScaleId,
            s.year,
            s.score,
            s.created_at,
            s.updated_at
        )
    """)

    print("Đã MERGE dữ liệu mới vào bảng benchmark")

# =========================
# 6. VERIFY
# =========================

spark.table(table_name).show(5, truncate=False)
spark.table(table_name).groupBy("year").count().orderBy("year").show()


LOAD BẢNG BENCHMARK
Bảng nessie.silver_tables.benchmark đã tồn tại → dùng MERGE (upsert).
Đã MERGE dữ liệu mới vào bảng benchmark
+--------------------+--------+--------+--------------+-----------------+--------------+----+-----+--------------------------+--------------------------+
|benchmarkId         |schoolId|majorId |subjectGroupId|selectionMethodId|gradingScaleId|year|score|created_at                |updated_at                |
+--------------------+--------+--------+--------------+-----------------+--------------+----+-----+--------------------------+--------------------------+
|676054869222677887  |HHK     |7510302A|2             |2                |0             |2025|18.0 |2025-12-01 16:00:01.747504|2025-12-01 16:00:01.747504|
|-4276703068313169330|HIU     |7340122 |35            |2                |0             |2025|15.0 |2025-12-01 16:00:01.747504|2025-12-01 16:00:01.747504|
|-1430534741141047354|DCT     |7480201 |NULL          |5                |1             |2025|740.0|2

## 7. Load Bảng REGION

In [8]:
print("=" * 80)
print("LOAD BẢNG REGION")
print("=" * 80)

df_region = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/region.csv")
df_region_silver = df_region.select(
    lpad(col(df_region.columns[0]).cast("string"), 2, "0").alias("regionId"),  # Format thành 2 chữ số: "1" -> "01"
    col(df_region.columns[1]).cast("string").alias("regionName"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("regionId").isNotNull() & col("regionName").isNotNull()).dropDuplicates(["regionId"])

df_region_silver.writeTo("nessie.silver_tables.region").using("iceberg").createOrReplace()
print(f"Đã ghi {df_region_silver.count()} dòng vào region")

# Verify
spark.table("nessie.silver_tables.region").show(10, truncate=False)

LOAD BẢNG REGION
Đã ghi 64 dòng vào region
+--------+-----------------------+-------------------------+-------------------------+
|regionId|regionName             |created_at               |updated_at               |
+--------+-----------------------+-------------------------+-------------------------+
|01      |Sở GDĐT Hà Nội         |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|02      |Sở GDĐT TP. Hồ Chí Minh|2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|03      |Sở GDĐT Hải Phòng      |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|04      |Sở GDĐT Đà Nẵng        |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|05      |Sở GDĐT Hà Giang       |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|06      |Sở GDĐT Cao Bằng       |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|07      |Sở GDĐT Lai Châu       |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|
|08      |Sở GDĐT Lào Cai        |2025-12-01 15:39:24.67929|2025-12-01 15:39:24.67929|


## 8. Load Bảng STUDENT_SCORES

In [22]:
from pyspark.sql.functions import (
    col, trim, regexp_replace, current_timestamp, lit,
    concat, substring, udf, input_file_name, regexp_extract
)
from pyspark.sql.types import MapType, IntegerType, DoubleType
from typing import Dict

print("=" * 80)
print("LOAD BẢNG STUDENT_SCORES - INCREMENTAL BY FILE (DELETE + APPEND)")
print("=" * 80)

# =====================================================
# 0. TẠO BẢNG LOG INGEST (LƯU FILE ĐÃ XỬ LÝ) NẾU CHƯA CÓ
# =====================================================
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.student_scores_ingest_log (
    path STRING,
    year INT,
    processed_at TIMESTAMP
) USING iceberg
""")

# =====================================================
# 1. LẤY DANH SÁCH TẤT CẢ FILE CSV HIỆN CÓ TRONG BRONZE
#    + TRỪ ĐI NHỮNG FILE ĐÃ INGEST (log)
# =====================================================

df_files = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.csv")
    .load("s3a://bronze/structured_data/điểm từng thí sinh/*/*.csv")
    .select("path")
)

df_log = spark.table("nessie.silver_tables.student_scores_ingest_log")

df_new_files = df_files.join(df_log, on="path", how="left_anti")
new_files = [r.path for r in df_new_files.collect()]

if not new_files:
    print("❌ Không có file mới nào, dừng job.")
else:
    print(f"✅ Phát hiện {len(new_files)} file mới cần xử lý.")

    # =====================================================
    # 2. ĐỌC CHỈ CÁC FILE MỚI + THÊM CỘT YEAR
    # =====================================================

    df_scores_raw = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(new_files)
        .withColumn("path", input_file_name())
    )

    df_scores_raw = df_scores_raw.withColumn(
        "Year",
        regexp_extract(col("path"), r"/(\d{4})/", 1).cast("int")
    )

    # =====================================================
    # 3. LOAD LOOKUP MÔN HỌC
    # =====================================================

    df_subject_lookup = spark.table("nessie.silver_tables.subject").select("subjectId", "subjectName")
    subject_map = {row.subjectName: row.subjectId for row in df_subject_lookup.collect()}
    print(f"\nĐã load {len(subject_map)} môn học để mapping")

    # =====================================================
    # 4. UDF PARSE ĐIỂM → Map<subjectId, score>
    # =====================================================

    def parse_scores_with_subject_id(score_string: str) -> Dict[int, float]:
        if not score_string or score_string.strip() == "":
            return {}
        scores_dict = {}
        try:
            pairs = score_string.split(",")
            for pair in pairs:
                if ":" in pair:
                    subject_name, score = pair.split(":")
                    subject_name = subject_name.strip()
                    # Map tên môn -> subjectId
                    if subject_name in subject_map:
                        subject_id = subject_map[subject_name]
                        try:
                            scores_dict[subject_id] = float(score.strip())
                        except:
                            pass
        except:
            pass
        return scores_dict

    parse_scores_udf = udf(parse_scores_with_subject_id, MapType(IntegerType(), DoubleType()))

    # =====================================================
    # 5. TRANSFORM → DATAFRAME STAGING (KHÔNG MERGE)
    # =====================================================

    # 1️⃣ Biến đầy đủ để append vào silver
    df_student_scores_stage = (
        df_scores_raw
        .withColumn("studentId", concat(col("SBD"), col("Year").cast("string")))
        .withColumn("scores", parse_scores_udf(col("DiemThi")))   # UDF ở đây
        .withColumn("regionId", substring(col("SBD"), 1, 2).cast("string"))
        .select(
            col("studentId").cast("string"),
            col("regionId").cast("string"),
            col("Year").cast("int").alias("year"),
            col("scores")
        )
        .filter(
            col("studentId").isNotNull() &
            col("year").isNotNull() &
            col("scores").isNotNull()
        )
    )
    
    # 2️⃣ Biến thứ hai chỉ có studentId — KHÔNG UDF → dùng để DELETE
    df_student_ids = (
        df_scores_raw
        .withColumn("studentId", concat(col("SBD"), col("Year").cast("string")))
        .select("studentId")
        .filter(col("studentId").isNotNull())
        # .dropDuplicates(["studentId"])
    )
    
    df_student_ids.createOrReplaceTempView("student_scores_new_ids")


    staging_count = df_student_scores_stage.count()
    print(f"Staging có {staging_count:,} dòng.")

    table_name = "nessie.silver_tables.student_scores"

        # =====================================================
    # 6. XOÁ studentId CŨ BẰNG CÁCH COLLECT RA PYTHON + DELETE IN (...)
    # =====================================================

    # Lấy list studentId distinct trong batch mới
    new_ids = [
        row.studentId
        for row in df_student_scores_stage.select("studentId").distinct().collect()
    ]

    print(f"Số studentId distinct trong batch mới: {len(new_ids):,}")

    # Kiểm tra bảng silver đã tồn tại chưa
    try:
        spark.table(table_name)
        table_exists = True
        print(f"Bảng {table_name} đã tồn tại → DELETE theo list studentId + APPEND.")
    except Exception:
        table_exists = False
        print(f"Bảng {table_name} chưa tồn tại → tạo mới từ batch, không cần xoá.")

    silver_count = spark.table(table_name).count() if table_exists else 0
    print(f"Số dòng trong bảng silver hiện tại: {silver_count:,}")

    if not table_exists:
        # 1️⃣ BẢNG CHƯA TỒN TẠI → TẠO MỚI
        (
            df_student_scores_stage
            .withColumn("created_at", current_timestamp())
            .withColumn("updated_at", current_timestamp())
            .writeTo(table_name)
            .using("iceberg")
            .createOrReplace()
        )
        print(f"✅ Đã tạo mới bảng {table_name} với {staging_count:,} dòng.")
    
    elif silver_count == 0:
        # 2️⃣ BẢNG TỒN TẠI NHƯNG RỖNG → KHÔNG XOÁ, CHỈ APPEND
        print("⚠️ Bảng silver đã tồn tại nhưng rỗng → chỉ append, không xoá.")
    
        (
            df_student_scores_stage
            .withColumn("created_at", current_timestamp())
            .withColumn("updated_at", current_timestamp())
            .writeTo(table_name)
            .using("iceberg")
            .append()
        )
        print(f"✅ Đã append {staging_count:,} dòng mới vào {table_name}.")
    
    elif new_ids:
        # 3️⃣ BẢNG TỒN TẠI VÀ new_ids KHÔNG RỖNG → DELETE + APPEND
        print("Bảng silver có dữ liệu → DELETE + APPEND.")
    
        chunk_size = 500
        from math import ceil
    
        num_chunks = ceil(len(new_ids) / chunk_size)
        print(f"Chia studentId thành {num_chunks} chunk để xoá...")
    
        for i in range(num_chunks):
            chunk = new_ids[i * chunk_size:(i + 1) * chunk_size]
            escaped_ids = [sid.replace("'", "''") for sid in chunk]
            in_list = ",".join([f"'{sid}'" for sid in escaped_ids])
    
            sql_delete = f"""
                DELETE FROM {table_name}
                WHERE studentId IN ({in_list})
            """
            spark.sql(sql_delete)
    
        print("✅ Đã xoá xong các studentId cũ trong silver.")
    
        (
            df_student_scores_stage
            .withColumn("created_at", current_timestamp())
            .withColumn("updated_at", current_timestamp())
            .writeTo(table_name)
            .using("iceberg")
            .append()
        )
        print(f"✅ Đã append {staging_count:,} dòng mới.")
    
    else:
        # 4️⃣ new_ids rỗng → không xoá, không append
        print("⚠️ Batch mới không có studentId nào hợp lệ → không làm gì cả.")


    # =====================================================
    # 7. GHI LOG FILE ĐÃ XỬ LÝ
    # =====================================================

    from pyspark.sql.functions import array, explode

    df_new_files_log = (
        df_new_files
        .withColumn("year", regexp_extract(col("path"), r"/(\d{4})/", 1).cast("int"))
        .withColumn("processed_at", current_timestamp())
    )

    (
        df_new_files_log
        .writeTo("nessie.silver_tables.student_scores_ingest_log")
        .using("iceberg")
        .append()
    )

    print(f"Đã ghi log {df_new_files_log.count():,} file đã xử lý.")

    # =====================================================
    # 8. VERIFY
    # =====================================================

    print("\nMẫu dữ liệu student_scores:")
    spark.table(table_name).show(5, truncate=False)
    spark.table(table_name).groupBy("year").count().orderBy("year").show()


LOAD BẢNG STUDENT_SCORES - INCREMENTAL BY FILE (DELETE + APPEND)
❌ Không có file mới nào, dừng job.


## 9. Load Bảng ARTICLE và COMMENT từ TikTok Data

In [2]:
import re, csv, io
from datetime import datetime

import pyspark.sql.functions as F
from pyspark.sql.functions import input_file_name, col, trim, current_timestamp
from pyspark.sql.types import (
    StructType, StructField, IntegerType, StringType, TimestampType
)
from pyspark.sql.utils import AnalysisException

# ===== Helpers =====
def clean_url(url: str):
    """Bỏ prefix 'https://www.tiktok.com/' trong urlUser"""
    if not url:
        return url
    prefix = "https://www.tiktok.com/"
    if url.startswith(prefix):
        return url[len(prefix):]
    return url

def clean_reply_to(name: str):
    """Bỏ suffix '?lang=vi-VN' trong replyTo"""
    if not name:
        return name
    suffix = "?lang=vi-VN"
    if name.endswith(suffix):
        return name[:-len(suffix)]
    return name

print("=" * 80)
print("LOAD BẢNG ARTICLE VÀ COMMENT TỪ TIKTOK DATA (INCREMENTAL)")
print("=" * 80)

# ===== 0. Tạo bảng log file nếu chưa có =====
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.tiktok_comment_files_log (
  file_path STRING,
  load_time TIMESTAMP
) USING iceberg
""")

# ===== 1. Xác định các file comment MỚI =====
tiktok_path = "s3a://bronze/MangXaHoi/tiktok-data/comments/*.csv"
print(f"Đọc danh sách file từ: {tiktok_path}")

df_all_files = (
    spark.read
         .option("header", "false")
         .option("encoding", "UTF-8")
         .csv(tiktok_path)
         .select(input_file_name().alias("file_path"))
         .distinct()
)

try:
    df_processed = spark.table("nessie.silver_tables.tiktok_comment_files_log") \
                        .select("file_path").distinct()
except AnalysisException:
    df_processed = spark.createDataFrame([], "file_path STRING")

df_new_files = df_all_files.join(df_processed, "file_path", "left_anti")

tiktok_files = [r.file_path for r in df_new_files.collect()]
print(f"Số file comment mới: {len(tiktok_files)}")

if not tiktok_files:
    print("Không có file comment mới, bỏ qua phần load comment.")
else:
    # ===== 2. Lấy max articleID & commentID hiện có =====
    try:
        max_article_id = spark.table("nessie.silver_tables.article") \
                              .agg(F.max("articleID")).collect()[0][0]
        if max_article_id is None:
            max_article_id = 0
    except AnalysisException:
        max_article_id = 0

    try:
        max_comment_id = spark.table("nessie.silver_tables.comment") \
                              .agg(F.max("commentID")).collect()[0][0]
        if max_comment_id is None:
            max_comment_id = 0
    except AnalysisException:
        max_comment_id = 0

    article_counter = max_article_id
    comment_counter = max_comment_id

    print(f"max articleID hiện có: {article_counter}")
    print(f"max commentID hiện có: {comment_counter}")

    # ===== Schema article & comment =====
    article_schema = StructType([
        StructField("articleID", IntegerType(), False),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("author", StringType(), True),
        StructField("url", StringType(), True),
        StructField("timePublish", TimestampType(), True),
        StructField("likeCount", IntegerType(), True),
        StructField("commentCount", IntegerType(), True),
        StructField("shareCount", IntegerType(), True),
        StructField("type", StringType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])

    comment_schema = StructType([
        StructField("commentID", IntegerType(), False),
        StructField("articleID", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("tagName", StringType(), True),
        StructField("urlUser", StringType(), True),
        StructField("comment", StringType(), True),
        StructField("commentTime", TimestampType(), True),
        StructField("commentLike", IntegerType(), True),
        StructField("levelComment", IntegerType(), True),
        StructField("replyTo", StringType(), True),
        StructField("numberOfReply", IntegerType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])

    total_article_new = 0
    total_comment_new = 0

    # ===== 3. Xử lý từng file comment MỚI =====
    for file_path in tiktok_files:
        try:
            print(f"\nXử lý file: {file_path}")

            # Đọc raw để lấy metadata
            df_raw = (spark.read
                      .option("header", "false")
                      .option("encoding", "UTF-8")
                      .csv(file_path))
            rows = df_raw.collect()

            # ----- Parse metadata bài post -----
            post_url = ""
            author = ""
            tag_name = ""
            author_url = ""
            time_publish = ""
            like_count = 0
            comment_count_meta = 0
            share_count = 0
            title = ""

            for r in rows[:20]:
                line = r[0] if r[0] else ""

                if "Post URL:" in line:
                    post_url = line.split("Post URL:")[1].strip()
                elif "Người đăng:" in line:
                    author = line.split("Người đăng:")[1].strip()
                elif "Tag người đăng:" in line:
                    tag_name = line.split("Tag người đăng:")[1].strip()
                elif "URL người đăng:" in line:
                    author_url = line.split("URL người đăng:")[1].strip()
                elif "Thời gian đăng:" in line:
                    time_str = line.split("Thời gian đăng:")[1].strip()
                    try:
                        time_publish = datetime.strptime(time_str, "%d-%m-%Y")
                    except:
                        time_publish = datetime.now()
                elif "Số lượt tym:" in line:
                    tym_str = line.split("Số lượt tym:")[1].strip()
                    try:
                        if "K" in tym_str:
                            like_count = int(float(tym_str.replace("K", "")) * 1000)
                        else:
                            like_count = int(tym_str)
                    except:
                        like_count = 0
                elif "Số lượt comment:" in line:
                    try:
                        c_str = line.split("Số lượt comment:")[1].strip()
                        if "K" in c_str:
                            comment_count_meta = int(float(c_str.replace("K", "")) * 1000)
                        else:
                            comment_count_meta = int(c_str)
                    except:
                        comment_count_meta = 0
                elif "Số lượt share:" in line:
                    share_str = line.split("Số lượt share:")[1].strip()
                    try:
                        if share_str != "N/A":
                            if "K" in share_str:
                                share_count = int(float(share_str.replace("K", "")) * 1000)
                            else:
                                share_count = int(share_str)
                        else:
                            share_count = 0
                    except:
                        share_count = 0
                elif "Mô tả của bài đăng:" in line:
                    first_part = line.split("Mô tả của bài đăng:")[1].strip()

                    if first_part.endswith('"') and first_part.count('"') >= 2:
                        title = first_part.strip('"')
                    else:
                        desc_lines = []
                        if first_part != "":
                            desc_lines.append(first_part.lstrip('"'))

                        j = rows.index(r) + 1
                        while j < len(rows):
                            next_line = rows[j][0] if rows[j][0] else ""
                            desc_lines.append(next_line)
                            if next_line.endswith('"'):
                                break
                            j += 1

                        full_desc = "\n".join(desc_lines).strip()
                        title = full_desc.strip('"').strip()

            video_url = clean_url(post_url)

            # ----- Xác định articleID (cũ → dùng lại, mới → tăng counter) -----
            try:
                df_article_exist = (spark.table("nessie.silver_tables.article")
                                         .select("articleID", "url")
                                         .filter(col("url") == video_url))
                exist_rows = df_article_exist.collect()
            except AnalysisException:
                exist_rows = []

            if exist_rows:
                article_id = exist_rows[0].articleID
                is_new_article = False
                print(f"Video {video_url} đã có articleID={article_id} → xoá comment cũ rồi insert lại")
                # Xoá comment cũ
                spark.sql(f"""
                    DELETE FROM nessie.silver_tables.comment
                    WHERE articleID = {article_id}
                """)
            else:
                article_counter += 1
                article_id = article_counter
                is_new_article = True
                print(f"Video {video_url} là mới → tạo articleID={article_id}")

            # ----- Nếu là article mới → insert vào bảng article -----
            if is_new_article:
                article_data = {
                    "articleID": article_id,
                    "title": title,
                    "description": None,   # Description sẽ được cập nhật sau từ sub
                    "author": author,
                    "url": video_url,
                    "timePublish": time_publish,
                    "likeCount": like_count,
                    "commentCount": comment_count_meta,
                    "shareCount": share_count,
                    "type": "TikTok",
                    "created_at": datetime.now(),
                    "updated_at": datetime.now()
                }
                df_article_one = spark.createDataFrame([article_data], article_schema)
                (df_article_one.writeTo("nessie.silver_tables.article")
                               .using("iceberg")
                               .append())
                total_article_new += 1

            # ----- Đọc lại file dạng text để parse comment chuẩn CSV -----
            df_text = spark.read.text(file_path)
            text_lines = [r.value if r.value is not None else "" for r in df_text.collect()]

            header_row_index = -1
            header_line = None

            for i, line in enumerate(text_lines):
                line_norm = line.strip()
                if (
                    line_norm.startswith("STT,")
                    and "Tên" in line_norm
                    and "Tag tên" in line_norm
                    and "URL" in line_norm
                    and "Comment" in line_norm
                    and "Time" in line_norm
                    and "Likes" in line_norm
                    and "Level Comment" in line_norm
                    and "Replied To Tag Name" in line_norm
                    and "Number of Replies" in line_norm
                ):
                    header_row_index = i
                    header_line = line_norm
                    break

            if header_row_index >= 0:
                print(f"  Tìm thấy header comment ở dòng {header_row_index}")
                print(f"  Header: {header_line}")

                csv_lines = text_lines[header_row_index:]
                csv_str = "\n".join(csv_lines)

                reader = csv.DictReader(io.StringIO(csv_str))
                comment_dict_rows = list(reader)
                print(f"  Tìm thấy {len(comment_dict_rows)} comment (sau khi parse CSV)")

                file_comments = []

                for r in comment_dict_rows:
                    try:
                        comment_time_str = r.get("Time", "") or ""
                        try:
                            comment_time = datetime.strptime(comment_time_str, "%d-%m-%Y")
                        except:
                            comment_time = datetime.now()

                        level_comment = 2 if (r.get("Level Comment") == "Yes") else 1

                        reply_to_raw = r.get("Replied To Tag Name", "")
                        if reply_to_raw and reply_to_raw != "---":
                            reply_to = clean_reply_to(reply_to_raw)
                        else:
                            reply_to = None

                        likes_raw = r.get("Likes", "")
                        comment_like = int(likes_raw) if (likes_raw and str(likes_raw).isdigit()) else 0

                        num_reply_raw = r.get("Number of Replies", "")
                        number_of_reply = int(num_reply_raw) if (num_reply_raw and str(num_reply_raw).isdigit()) else 0

                        comment_counter += 1
                        comment_data = {
                            "commentID": comment_counter,
                            "articleID": article_id,
                            "name": r.get("Tên", "") or "",
                            "tagName": r.get("Tag tên", "") or "",
                            "urlUser": clean_url(r.get("URL", "") or ""),
                            "comment": r.get("Comment", "") or "",
                            "commentTime": comment_time,
                            "commentLike": comment_like,
                            "levelComment": level_comment,
                            "replyTo": reply_to,
                            "numberOfReply": number_of_reply,
                            "created_at": datetime.now(),
                            "updated_at": datetime.now()
                        }
                        file_comments.append(comment_data)
                    except Exception as e:
                        print(f"  Lỗi xử lý comment: {e}")
                        continue

                # if file_comments:
                #     # tạo DF từ list
                #     df_comment_batch = spark.createDataFrame(file_comments, comment_schema)
                
                #     # CHIA NHỎ ra nhiều partition để mỗi task ghi ít dữ liệu hơn
                #     df_comment_batch = df_comment_batch.repartition(16)  # thử 4, nếu vẫn OOM thì tăng 8, 16
                
                #     # số bản ghi đã biết sẵn từ list, không cần .count()
                #     # written = len(file_comments)
                #     written = 0
                
                #     (df_comment_batch.writeTo("nessie.silver_tables.comment")
                #                      .using("iceberg")
                #                      .append())
                
                #     total_comment_new += written
                #     print(f"  Đã ghi {written} comment mới cho articleID={article_id}")
                # else:
                #     print("  File này không có comment nào để ghi")
                # chỗ này nên sửa lại nếu chạy trên spark cluster ( hiện đang tối ưu nhỏ hơn để chạy trên jupyter)
                if file_comments:
                    # CHUNK_SIZE: số comment tối đa mỗi lần ghi
                    CHUNK_SIZE = 1000  # nếu vẫn OOM thì giảm còn 2000 hoặc 1000
                
                    total_for_article = 0
                
                    for i in range(0, len(file_comments), CHUNK_SIZE):
                        chunk = file_comments[i:i + CHUNK_SIZE]
                
                        # Tạo DataFrame cho chunk hiện tại
                        df_chunk = spark.createDataFrame(chunk, comment_schema)
                
                        # Chia nhỏ thêm trong JVM (nếu muốn)
                        df_chunk = df_chunk.repartition(2)  # có thể thử 2 / 4 / 8
                
                        # Ghi chunk vào Iceberg
                        (df_chunk.writeTo("nessie.silver_tables.comment")
                                .using("iceberg")
                                .append())
                
                        total_for_article += len(chunk)
                        print(f"  Đã ghi thêm {len(chunk)} comment cho articleID={article_id} (tổng tạm: {total_for_article})")
                
                    total_comment_new += total_for_article
                    print(f"  ==> Tổng cộng ghi {total_for_article} comment cho articleID={article_id}")
                else:
                    print("  File này không có comment nào để ghi")

            else:
                print("  Không tìm thấy header comment trong file")

            # ----- Ghi log file đã xử lý -----
            df_log = spark.createDataFrame(
                [(file_path, datetime.now())],
                "file_path STRING, load_time TIMESTAMP"
            )
            (df_log.writeTo("nessie.silver_tables.tiktok_comment_files_log")
                  .using("iceberg")
                  .append())

            print(f"Đã xử lý xong file {file_path}")

        except Exception as e:
            print(f"Lỗi xử lý file {file_path}: {e}")
            import traceback
            traceback.print_exc()
            continue

    print("\n" + "=" * 80)
    print(f"TỔNG ARTICLE MỚI:  {total_article_new}")
    print(f"TỔNG COMMENT MỚI:  {total_comment_new}")
    print("=" * 80)


LOAD BẢNG ARTICLE VÀ COMMENT TỪ TIKTOK DATA (INCREMENTAL)
Đọc danh sách file từ: s3a://bronze/MangXaHoi/tiktok-data/comments/*.csv


Số file comment mới: 1700
max articleID hiện có: 70
max commentID hiện có: 100357

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-09-30T10-05-17.csv
Video @thaihoa_entertainment/video/7529828371610488072 đã có articleID=70 → xoá comment cũ rồi insert lại
  Tìm thấy header comment ở dòng 17
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 658 comment (sau khi parse CSV)


  Đã ghi thêm 658 comment cho articleID=70 (tổng tạm: 658)
  ==> Tổng cộng ghi 658 comment cho articleID=70


Đã xử lý xong file s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-09-30T10-05-17.csv

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-11-06T01-07-16.csv
Video @nh_gem/video/7382458058544336145 là mới → tạo articleID=71
  Tìm thấy header comment ở dòng 16
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 507 comment (sau khi parse CSV)


  Đã ghi thêm 507 comment cho articleID=71 (tổng tạm: 507)
  ==> Tổng cộng ghi 507 comment cho articleID=71
Đã xử lý xong file s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-11-06T01-07-16.csv

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-09-28T11-25-59.csv
Video @vietnamstats.net/video/7374874369736133921 là mới → tạo articleID=72
  Tìm thấy header comment ở dòng 16
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 518 comment (sau khi parse CSV)


...

  Đã ghi thêm 73 comment cho articleID=424 (tổng tạm: 73)
  ==> Tổng cộng ghi 73 comment cho articleID=424
Đã xử lý xong file s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-10-02T15-04-09.csv

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-09-18T01-04-17.csv
Video @ngatichcuc/video/7212107157091765530 là mới → tạo articleID=425
  Tìm thấy header comment ở dòng 16
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 86 comment (sau khi parse CSV)


  Đã ghi thêm 86 comment cho articleID=425 (tổng tạm: 86)
  ==> Tổng cộng ghi 86 comment cho articleID=425
Đã xử lý xong file s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-09-18T01-04-17.csv

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-10-29T17-50-25.csv
Video @hayonline24h/video/6885625088654314753 là mới → tạo articleID=426
  Tìm thấy header comment ở dòng 16
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 89 comment (sau khi parse CSV)


  Đã ghi thêm 89 comment cho articleID=426 (tổng tạm: 89)
  ==> Tổng cộng ghi 89 comment cho articleID=426
Đã xử lý xong file s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-10-29T17-50-25.csv

Xử lý file: s3a://bronze/MangXaHoi/tiktok-data/comments/tiktok_comments_2025-11-07T00-32-07.csv
Video @hunre.uni/video/6990335452523826459 là mới → tạo articleID=427
  Tìm thấy header comment ở dòng 16
  Header: STT,Tên,Tag tên,URL,Comment,Time,Likes,Level Comment,Replied To Tag Name,Number of Replies
  Tìm thấy 114 comment (sau khi parse CSV)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
[Stage 4784:>                                                       (0 + 1) / 1]

KeyboardInterrupt: 

In [ ]:
# # Tạo DataFrame cho article
# if all_articles:
#     article_schema = StructType([
#         StructField("articleID", IntegerType(), False),
#         StructField("title", StringType(), True),
#         StructField("description", StringType(), True),
#         StructField("author", StringType(), True),
#         StructField("url", StringType(), True),
#         StructField("timePublish", TimestampType(), True),
#         StructField("likeCount", IntegerType(), True),
#         StructField("commentCount", IntegerType(), True),
#         StructField("shareCount", IntegerType(), True),
#         StructField("type", StringType(), True),
#         StructField("created_at", TimestampType(), True),
#         StructField("updated_at", TimestampType(), True)
#     ])
    
#     df_article_silver = spark.createDataFrame(all_articles, article_schema)
    
#     # Ghi vào bảng article
#     df_article_silver.writeTo("nessie.silver_tables.article").using("iceberg").createOrReplace()
#     print(f"Đã ghi {df_article_silver.count()} dòng vào bảng article")
    
#     # Verify
#     # spark.table("nessie.silver_tables.article").show(5, truncate=False)
# else:
#     print("Không có dữ liệu article để ghi")

In [ ]:
# # Tạo DataFrame cho comment
# if all_comments:
#     comment_schema = StructType([
#         StructField("commentID", IntegerType(), False),
#         StructField("articleID", IntegerType(), True),
#         StructField("name", StringType(), True),
#         StructField("tagName", StringType(), True),
#         StructField("urlUser", StringType(), True),
#         StructField("comment", StringType(), True),
#         StructField("commentTime", TimestampType(), True),
#         StructField("commentLike", IntegerType(), True),
#         StructField("levelComment", IntegerType(), True),
#         StructField("replyTo", StringType(), True),
#         StructField("numberOfReply", IntegerType(), True),
#         StructField("created_at", TimestampType(), True),
#         StructField("updated_at", TimestampType(), True)
#     ])
    
#     df_comment_silver = spark.createDataFrame(all_comments, comment_schema)
    
#     # Ghi vào bảng comment
#     df_comment_silver.writeTo("nessie.silver_tables.comment").using("iceberg").createOrReplace()
#     print(f"Đã ghi {df_comment_silver.count()} dòng vào bảng comment")
    
#     # Verify
#     spark.table("nessie.silver_tables.comment").show(5, truncate=False)
    
#     # Thống kê theo article
#     print("\nThống kê comment theo article:")
#     spark.table("nessie.silver_tables.comment").groupBy("articleID").count().orderBy("articleID").show()
# else:
#     print("Không có dữ liệu comment để ghi")

In [ ]:
#### Load sub vào cột description cho tiktok posts

In [ ]:
print("=" * 80)
print("CẬP NHẬT TITLE & DESCRIPTION CHO BẢNG ARTICLE TỪ FILE SUB (INCREMENTAL)")
print("=" * 80)

# Tạo bảng log file sub nếu chưa có
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.tiktok_sub_files_log (
  file_path STRING,
  load_time TIMESTAMP
) USING iceberg
""")

SUB_PATH = "s3a://bronze/MangXaHoi/tiktok-data/sub/*.csv"

# ----- Lấy danh sách file sub hiện có -----
df_all_sub_files = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "false")
         .option("encoding", "UTF-8")
         .csv(SUB_PATH)
         .select(input_file_name().alias("file_path"))
         .distinct()
)

try:
    df_sub_processed = spark.table("nessie.silver_tables.tiktok_sub_files_log") \
                            .select("file_path").distinct()
except AnalysisException:
    df_sub_processed = spark.createDataFrame([], "file_path STRING")

df_new_sub_files = df_all_sub_files.join(df_sub_processed, "file_path", "left_anti")
new_sub_files = [r.file_path for r in df_new_sub_files.collect()]

print(f"Số file sub mới: {len(new_sub_files)}")

if not new_sub_files:
    print("Không có file sub mới, bỏ qua bước cập nhật title/description.")
else:
    # ----- Đọc data từ các file sub MỚI -----
    df_desc_raw = (
        spark.read
             .option("header", "true")
             .option("inferSchema", "false")
             .option("encoding", "UTF-8")
             .csv(new_sub_files)
    )

    print(f"Đọc được {df_desc_raw.count():,} dòng từ các file sub mới")

    # Chuẩn hoá cột url, title, description
    # new_title: thường là caption / mô tả ngắn
    # new_description: toàn bộ sub
    df_desc = (
        df_desc_raw
            .select(
                trim(col("url")).alias("url"),
                trim(col("description")).alias("new_title"),
                trim(col("sub")).alias("new_description")
            )
            .filter(col("url").isNotNull() & (col("url") != ""))
    )

    print(f"Số dòng dùng để cập nhật: {df_desc.count():,}")

    df_desc.createOrReplaceTempView("article_desc_update")

    # ----- CHỈ UPDATE các article đã tồn tại, KHÔNG insert mới -----
    spark.sql("""
    MERGE INTO nessie.silver_tables.article AS a
    USING article_desc_update AS d
    ON a.url = d.url
    WHEN MATCHED THEN UPDATE SET
      a.title       = d.new_title,
      a.description = d.new_description,
      a.updated_at  = current_timestamp()
    """)

    print("Đã cập nhật title & description cho các article có url trùng trong bảng article!")

    # Ghi log các file sub đã xử lý
    df_sub_log = spark.createDataFrame(
        [(p, datetime.now()) for p in new_sub_files],
        "file_path STRING, load_time TIMESTAMP"
    )
    (df_sub_log.writeTo("nessie.silver_tables.tiktok_sub_files_log")
              .using("iceberg")
              .append())


In [ ]:
#### Load Facebook data

In [13]:
import re
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Map tháng tiếng Việt -> số
MONTH_MAP = {
    "Tháng 1": "01", "Tháng 2": "02", "Tháng 3": "03", "Tháng 4": "04",
    "Tháng 5": "05", "Tháng 6": "06", "Tháng 7": "07", "Tháng 8": "08",
    "Tháng 9": "09", "Tháng 10": "10", "Tháng 11": "11", "Tháng 12": "12"
}

def parse_vietnam_datetime(dt_str):
    """
    Convert:
    'Thứ Sáu, 1 Tháng 8, 2025 lúc 20:18'
          ↓
    '2025-08-01 20:18:00'
    """
    if not dt_str:
        return None
    
    try:
        # Bỏ tiền tố thứ ngày
        # Ví dụ: "Thứ Sáu, " → ""
        dt_str = dt_str.split(",", 1)[1].strip()

        # dt_str còn lại:
        # "1 Tháng 8, 2025 lúc 20:18"

        # Tách ngày – tháng tiếng Việt
        # 1 Tháng 8
        match = re.search(r"(\d+)\s+(Tháng\s+\d+)", dt_str)
        if not match:
            return None

        day = match.group(1)
        month_text = match.group(2)
        month = MONTH_MAP.get(month_text)

        # Lấy năm
        year_match = re.search(r",\s*(\d{4})", dt_str)
        if not year_match:
            return None
        year = year_match.group(1)

        # Lấy giờ phút
        time_match = re.search(r"lúc\s+(\d{1,2}:\d{2})", dt_str)
        if time_match:
            time_str = time_match.group(1)
        else:
            time_str = "00:00"

        # Format chuẩn: yyyy-MM-dd HH:mm:ss
        final_str = f"{year}-{month}-{int(day):02d} {time_str}:00"
        return final_str

    except Exception:
        return None


parse_vn_time_udf = udf(parse_vietnam_datetime, StringType())


In [11]:
# spark.sql("""
# DELETE FROM nessie.silver_tables.article
# WHERE type = 'facebook'
# """)


DataFrame[]

In [12]:
# spark.sql("""
# DELETE FROM nessie.silver_tables.comment
# WHERE articleID NOT IN (
#     SELECT articleID FROM nessie.silver_tables.article
# )
# """)


DataFrame[]

In [14]:
from pyspark.sql import Window
from pyspark.sql.functions import (
    col, trim, current_timestamp, to_timestamp,
    row_number, lit, max as F_max
)

print("=" * 80)
print("LOAD DỮ LIỆU FACEBOOK VÀO BẢNG ARTICLE & COMMENT (KHÔNG TRÙNG ID)")
print("=" * 80)

posts_path = "s3a://bronze/MangXaHoi/Face-data/posts_FB.csv"
comments_path = "s3a://bronze/MangXaHoi/Face-data/comments_FB.csv"

# ====================================================
# 0. LẤY MAX ID HIỆN CÓ TRONG BẢNG
# ====================================================
# Nếu bảng rỗng thì maxID = None -> dùng 0
try:
    max_article_row = (
        spark.table("nessie.silver_tables.article")
             .agg(F_max("articleID").alias("maxID"))
             .collect()[0]
    )
    base_article_id = max_article_row["maxID"] or 0
except Exception:
    base_article_id = 0

try:
    max_comment_row = (
        spark.table("nessie.silver_tables.comment")
             .agg(F_max("commentID").alias("maxID"))
             .collect()[0]
    )
    base_comment_id = max_comment_row["maxID"] or 0
except Exception:
    base_comment_id = 0

print(f"base_article_id (offset) = {base_article_id}")
print(f"base_comment_id (offset) = {base_comment_id}")

# ====================================================
# 1. ĐỌC BẢNG POSTS_FB -> ARTICLE
# ====================================================
df_posts_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(posts_path)
)

print(f"Đọc được {df_posts_raw.count():,} dòng từ posts_FB")

df_posts_clean = df_posts_raw.select(
    trim(col("ID")).alias("url_post"),      # URL bài viết (dùng để join)
    trim(col("Title")).alias("title"),
    trim(col("Description")).alias("description"),
    trim(col("Author")).alias("author"),
    trim(col("Url")).alias("url_extra"),
    trim(col("TimePublish")).alias("TimePublish_raw"),
    trim(col("Like")).alias("Like_raw"),
    trim(col("Share")).alias("Share_raw"),
    trim(col("Comment")).alias("Comment_raw")
)

df_posts_parsed = (
    df_posts_clean
        # Chuẩn hoá string tiếng Việt -> 'yyyy-MM-dd HH:mm:ss'
        .withColumn("TimePublish_clean", parse_vn_time_udf(col("TimePublish_raw")))
        # Convert string chuẩn -> timestamp
        .withColumn("timePublish", to_timestamp(col("TimePublish_clean")))
        .withColumn("likeCount", col("Like_raw").cast("int"))
        .withColumn("shareCount", col("Share_raw").cast("int"))
        .withColumn("commentCount", col("Comment_raw").cast("int"))
)


# Window để đánh số local trong batch FB
w_article = Window.orderBy("url_post")

df_article_silver = (
    df_posts_parsed
        # articleID_local: 1,2,3,... trong batch Facebook
        .withColumn("articleID_local", row_number().over(w_article))
        # articleID thật = offset + local
        .withColumn(
            "articleID",
            (col("articleID_local") + lit(base_article_id)).cast("int")
        )
        .select(
            col("articleID"),
            col("title"),
            col("description"),
            col("author"),
            col("url_post").alias("url"),
            col("timePublish"),
            col("likeCount"),
            col("commentCount"),
            col("shareCount"),
            lit("facebook").alias("type"),
            current_timestamp().alias("created_at"),
            current_timestamp().alias("updated_at")
        )
)

print("Mẫu dữ liệu article_silver:")
df_article_silver.show(5, truncate=False)

# Ghi vào bảng article
df_article_silver.writeTo("nessie.silver_tables.article").using("iceberg").append()
print(f"Đã ghi {df_article_silver.count():,} dòng vào nessie.silver_tables.article")

# Tạo lookup cho batch FB (chỉ cần url & articleID của batch này)
df_article_fb_lookup = df_article_silver.select("articleID", "url").distinct()

# ====================================================
# 2. ĐỌC BẢNG COMMENT_FB -> COMMENT, JOIN THEO URL
# ====================================================
df_cmt_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(comments_path)
)

print(f"Đọc được {df_cmt_raw.count():,} dòng từ comment_FB")

# comment_FB: STT,Id_post,Comment
df_cmt_clean = df_cmt_raw.select(
    trim(col("STT")).alias("stt"),
    trim(col("Id_post")).alias("post_url"),
    trim(col("Comment")).alias("comment_text")
).filter(col("post_url").isNotNull() & (col("post_url") != ""))

# JOIN comment với batch article FB vừa tạo (không join toàn bộ bảng để tránh nhầm nguồn khác)
df_cmt_joined = (
    df_cmt_clean.alias("c")
        .join(
            df_article_fb_lookup.alias("a"),
            col("c.post_url") == col("a.url"),
            "inner"
        )
)

print(f"Số dòng comment match với article FB: {df_cmt_joined.count():,}")

# Window để đánh số local comment trong batch FB
w_comment = Window.orderBy(col("a.articleID"), col("c.stt"))

df_comment_silver = (
    df_cmt_joined
        .withColumn("commentID_local", row_number().over(w_comment))
        .withColumn(
            "commentID",
            (col("commentID_local") + lit(base_comment_id)).cast("int")
        )
        .select(
            col("commentID"),
            col("a.articleID").alias("articleID"),
            lit(None).cast("string").alias("name"),
            lit(None).cast("string").alias("tagName"),
            lit(None).cast("string").alias("urlUser"),
            col("comment_text").alias("comment"),
            lit(None).cast("timestamp").alias("commentTime"),
            lit(None).cast("int").alias("commentLike"),
            lit(1).cast("int").alias("levelComment"),
            lit(None).cast("string").alias("replyTo"),
            lit(0).cast("int").alias("numberOfReply"),
            current_timestamp().alias("created_at"),
            current_timestamp().alias("updated_at")
        )
)

print("Mẫu dữ liệu comment_silver:")
df_comment_silver.show(5, truncate=False)

df_comment_silver.writeTo("nessie.silver_tables.comment").using("iceberg").append()
print(f"Đã ghi {df_comment_silver.count():,} dòng vào nessie.silver_tables.comment")

# ====================================================
# 3. CHECK LẠI
# ====================================================
print("\nCHECK lại article (facebook):")
spark.table("nessie.silver_tables.article").where("type = 'facebook'").show(5, truncate=False)

print("\nCHECK lại comment (facebook, join với article):")
spark.sql("""
SELECT c.commentID, c.articleID, a.title, c.comment
FROM nessie.silver_tables.comment c
JOIN nessie.silver_tables.article a
  ON c.articleID = a.articleID
WHERE a.type = 'facebook'
LIMIT 10
""").show(truncate=False)


LOAD DỮ LIỆU FACEBOOK VÀO BẢNG ARTICLE & COMMENT (KHÔNG TRÙNG ID)
base_article_id (offset) = 1770
base_comment_id (offset) = 210746
Đọc được 1,295 dòng từ posts_FB
Mẫu dữ liệu article_silver:
+---------+-----+----------------------------------------------------------------------------------------------------------+----------------------+----------------+-------------------+---------+------------+----------+--------+--------------------------+--------------------------+
|articleID|title|description                                                                                               |author                |url             |timePublish        |likeCount|commentCount|shareCount|type    |created_at                |updated_at                |
+---------+-----+----------------------------------------------------------------------------------------------------------+----------------------+----------------+-------------------+---------+------------+----------+--------+-----------------